# Feature Selection

In this short (compared to Feature Engineering one) notebook, I'll continue data preparation process. Here I'll try to "filter down" bloated dataset that we got at the end of previous notebook. Various techniques will be applied to select most useful for prediction features.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.compose import ColumnTransformer
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 150)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
sns.set()
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
import gc
from imports import *

In [14]:
def custom_lgbm_cv_small(features, target, k=5):
    metric_df=pd.DataFrame(columns=['Train AUC', 'Test AUC'])
    feat_importances=[]
    kfold=StratifiedKFold(k)
    for f, (tr, te) in enumerate(kfold.split(features, y=target)):
        X_train, y_train=features.iloc[tr, :], target.iloc[tr]
        X_test, y_test=features.iloc[te, :], target.iloc[te]

        weight=np.count_nonzero(y_train==0)/np.count_nonzero(y_train==1)

        params={'num_boost_round': 10000,
                'objective': 'binary',
                'scale_pos_weight': weight,
                'metric': 'auc',
                'learning_rate': 0.05,
                'reg_alpha': 0.1,
                'reg_lambda': 0.1,
                'subsample': 0.8,
                'n_jobs': -1,
                'random_state': 5,
                'verbose': -1}

        dtrain=lgb.Dataset(X_train, label=y_train)
        dval=lgb.Dataset(X_test, label=y_test)

        model=lgb.train(
                params=params,
                train_set=dtrain,
                valid_sets=[dtrain, dval],
                valid_names=['train', 'test'],
                callbacks=[lgb.early_stopping(100, verbose=-1)],
                verbose_eval=False)
        
        test_score, train_score=model.best_score['test']['auc'], model.best_score['train']['auc']
        metric_df.loc[f]=[train_score, test_score]

        feat_importances.append(model.feature_importance(importance_type='gain'))
    
    feat_importances=np.array(feat_importances).mean(axis=0)
    feat_importances_df=pd.DataFrame({'feature': features.columns,
                                        'importance': feat_importances})
    metric_df.loc['Avg']=[metric_df['Train AUC'].mean(), metric_df['Test AUC'].mean()]
    return metric_df, feat_importances_df

In [3]:
train=pd.read_csv('../data/full_train.csv')
train=convert_types(train, print_info=True)
train.shape

Original Memory Usage: 3.37 gb.
New Memory Usage: 1.71 gb.


(307511, 1369)

Note: with huge files like this, it may be a good idea to [use some formats other than csv](https://pythonspeed.com/articles/pandas-read-csv-fast/). It'll save you a lot if time. 

## Correlation

One of the simplest methods of feature selection is to remove highly correlated features. Presence of this features can negatively affect model's ability to learn, generalize. Hence, we should remove them. One should select the treshold themselves, but it's usually 0.8-0.9.

In [4]:
corr_mat=train.corr().abs()
upper=corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(np.bool))
upper.head()

<ipython-input-4-e9ccfe60ee12>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper=corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(np.bool))


,Unnamed: 0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NEW_AMT_CREDIT_TO_AMT_INCOME,NEW_AMT_CREDIT_TO_AMT_ANNUITY,NEW_AMT_CREDIT_TO_AMT_GOODS_PRICE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_CONTRACT_TYPE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_HOUSING_TYPE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_ORGANIZATION_TYPE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_EDUCATION_TYPE,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_GENDER,NEW_AMT_CREDIT_TO_MEAN_AMT_CREDIT_BY_FAMILY_STATUS,NEW_AMT_CREDIT_TO_MEAN_AMT_INCOME_BY_AGE_GROUP,NEW_AMT_INCOME_BY_AGE_GROUP,NEW_AMT_INCOME_BY_CNT_CHILD,NEW_AMT_INCOME_BY_CNT_FAM_MEMBERS,NEW_AMT_INCOME_BY_AGE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_CONTRACT_TYPE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_HOUSING_TYPE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_ORGANIZATION_TYPE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_EDUCATION_TYPE,NEW_AMT_INCOME_TO_MEAN_AMT_CREDIT_BY_GENDER,NEW_AMT_INCOME_TO_MEAN_AMT_INCOME_BY_AGE_GROUP,NEW_DOC_FLAG_MEAN,NEW_DOC_FLAG_SUM,NEW_CONTACT_FLAG_MEAN,NEW_CONTACT_FLAG_SUM,NEW_ADDRESS_FLAG_MEAN,NEW_ADDRESS_FLAG_SUM,NEW_OWN_CAR_REALTY_COMBINATION,NEW_AGE_TO_MEAN_AGE_BY_FLAG_OWN_REALTY,NEW_AGE_TO_MEAN_AGE_BY_FLAG_OWN_CAR,NEW_AGE_TO_MEAN_AGE_BY_HOUSING_TYPE,NEW_DAYS_EMPLOYED_TO_DAYS_BIRTH,NEW_DAYS_REGISTRATION_TO_DAYS_BIRTH,NEW_OWN_CAR_AGE_TO_DAYS_BIRTH,NEW_OWN_CAR_AGE_TO_DAYS_EMPLOYED,NEW_DAYS_LAST_PHONE_CHANGE_TO_DAYS_BIRTH,NEW_DAYS_LAST_PHONE_CHANGE_TO_DAYS_EMPLOYED,NEW_CNT_CHILD_TO_CNT_FAM_MEMBERS,NEW_EXT_SOURCES_MEAN,NEW_EXT_SOURCES_STD,NEW_DAYS_CHANGE_MEAN,NEW_REGION_RATING_CLIENT_MEAN,NEW_30_CNT_SOCIAL_CIRCLE_MEAN,NEW_60_CNT_SOCIAL_CIRCLE_MEAN,bureau_DAYS_CREDIT_sum,bureau_DAYS_ENDDATE_FACT_sum,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_ENDDATE_min,bureau_DAYS_ENDDATE_FACT_min,bureau_DAYS_ENDDATE_FACT_mean,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_UPDATE_sum,bureau_DAYS_ENDDATE_FACT_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_UPDATE_max,bureau_CNT_CREDIT_PROLONG_min,bureau_CREDIT_DAY_OVERDUE_min,bureau_AMT_CREDIT_SUM_OVERDUE_min,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,bureau_CREDIT_DAY_OVERDUE_sum,bureau

In [5]:
upper.to_csv('../data/corr_mat.csv')

I chose 90% colinearity as a threshold, you can try other values. Spoiler, lots of features are going to be dropped.

In [6]:
to_drop=[column for column in upper.columns if any(upper[column] > 0.9)]
print(f'Correlation: {len(to_drop)} will be removed')

Correlation: 620 will be removed


In [7]:
train=train.drop(to_drop, axis=1)
train.shape

(307511, 749)

In [8]:
gc.enable()
del corr_mat, upper
gc.collect()

15

In the very first step of FS, the dataset lost approx 40% of features. How will it affect modelling results?

In [9]:
X, y=train.drop('TARGET', axis=1), train['TARGET']
cat_cols, num_cols=X.select_dtypes(include=['category', 'object']).columns, X.select_dtypes('number').columns

ohe=OneHotEncoder(sparse=False, handle_unknown='ignore')
col_tr=ColumnTransformer([
    ('cat', ohe, cat_cols),
    ('num', 'passthrough', num_cols)
])

res, importances=custom_lgbm_cv(X, y, col_tr)
res

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[384]	train's auc: 0.86834	test's auc: 0.78448
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[366]	train's auc: 0.865536	test's auc: 0.788918
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[388]	train's auc: 0.869404	test's auc: 0.782447
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[282]	train's auc: 0.851089	test's auc: 0.78666
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[599]	train's auc: 0.896481	test's auc: 0.782378


,Train AUC,Test AUC
0,0.8683,0.7845
1,0.8655,0.7889
2,0.8694,0.7824
3,0.8511,0.7867
4,0.8965,0.7824
Avg,0.8702,0.7850


Well, there are much less features, training is faster and results are almost the same as with the full data. Great! 

## Missing features

Now I'll remove mostly empty features, i.e., features with 70-90% of NaNs. I'll re-use `miss_table` function from EDA notebook.

In [10]:
def miss_table(data):
    miss_table=data.isna().sum().to_frame(name='Count')
    miss_table['Percent']=miss_table['Count']/len(data)*100
    miss_table['Dtype']=data.dtypes[miss_table.index]
    miss_table['Count']=miss_table['Count'].replace({0: np.nan})
    miss_table=miss_table.dropna()
    print(f"There are {len(miss_table)}/{data.shape[1]} columns with missing values")
    print('Distribution by dtypes:')
    print(miss_table['Dtype'].value_counts())
    return miss_table.sort_values(by='Count', ascending=False)

mt=miss_table(train)
mt.head()

There are 689/749 columns with missing values
Distribution by dtypes:
float32     683
category      1
category      1
category      1
category      1
category      1
category      1
Name: Dtype, dtype: int64


,Count,Percent,Dtype
previous_RATE_INTEREST_PRIVILEGED_min,302902.0000,98.5012,float32
loan_card_AMT_PAYMENT_CURRENT_min_min,246451.0000,80.1438,float32
loan_card_AMT_DRAWINGS_ATM_CURRENT_max_min,246371.0000,80.1178,float32
loan_card_CNT_DRAWINGS_ATM_CURRENT_mean_min,246371.0000,80.1178,float32
loan_card_CNT_DRAWINGS_ATM_CURRENT_min_min,246371.0000,80.1178,float32


In [11]:
thresh=80
to_drop=mt.loc[mt['Percent']>thresh].index
print(f'{len(to_drop)} features with {thresh}% of NaNs will be removed')

16 features with 80% of NaNs will be removed


A few features got dropped, let's see modelling results.

In [12]:
train=train.drop(to_drop, axis=1)
train.shape

(307511, 733)

In [13]:
def quick_cv(df):
    X, y=df.drop('TARGET', axis=1), df['TARGET']
    cat_cols, num_cols=X.select_dtypes(include=['category', 'object']).columns, X.select_dtypes('number').columns

    ohe=OneHotEncoder(sparse=False, handle_unknown='ignore')
    col_tr=ColumnTransformer([
        ('cat', ohe, cat_cols),
        ('num', 'passthrough', num_cols)
    ])

    res, importances=custom_lgbm_cv(X, y, col_tr)
    return res, importances

In [14]:
res, importances=quick_cv(train)
res

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[377]	train's auc: 0.867565	test's auc: 0.785711
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[485]	train's auc: 0.882736	test's auc: 0.788393
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[434]	train's auc: 0.876578	test's auc: 0.783368
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[297]	train's auc: 0.853771	test's auc: 0.786835
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[492]	train's auc: 0.883617	test's auc: 0.783562


(     Train AUC  Test AUC
 0       0.8676    0.7857
 1       0.8827    0.7884
 2       0.8766    0.7834
 3       0.8538    0.7868
 4       0.8836    0.7836
 Avg     0.8729    0.7856,
                                              feature  importance
 0                 cat__NAME_CONTRACT_TYPE_Cash loans   1857.3655
 1            cat__NAME_CONTRACT_TYPE_Revolving loans      0.0000
 2                                 cat__CODE_GENDER_F  21172.4874
 3                                 cat__CODE_GENDER_M      0.0000
 4                                cat__FLAG_OWN_CAR_N     28.3209
 ..                                               ...         ...
 855  num__loan_card_AMT_CREDIT_LIMIT_ACTUAL_mean_min   2515.9813
 856   num__loan_card_AMT_CREDIT_LIMIT_ACTUAL_min_sum    885.5537
 857   num__loan_card_AMT_CREDIT_LIMIT_ACTUAL_sum_min   1392.7590
 858   num__loan_card_AMT_CREDIT_LIMIT_ACTUAL_max_sum   1444.0484
 859   num__loan_card_AMT_CREDIT_LIMIT_ACTUAL_sum_sum   1808.6986
 
 [860 rows x 2 columns]

Performance remains stable (roughly the same), even though we got less and les featues. Perfect!

## Low Variance

Features with zero or very low (here threshold is subjective) variance can be dropped too. Let's try it out.

In [35]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()
numeric_data=train.iloc[:, 1:].select_dtypes('number').reset_index()
numeric_data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [38]:
for i in numeric_data.columns:
    numeric_data[i].fillna(value=numeric_data[i].mean(), inplace=True) #replace NaN with mean of dimension
    numeric_data[i] = scaler.fit_transform(numeric_data[i].values.reshape(-1,1)) 

train_vars=numeric_data.var().sort_values()

In [55]:
to_drop=train_vars[train_vars<0.00005].index
to_drop.shape

(112,)

In [56]:
train=train.drop(to_drop, axis=1)
train.shape

(307511, 621)

In [57]:
res, importances=quick_cv(train)
res

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[322]	train's auc: 0.856435	test's auc: 0.784166
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[311]	train's auc: 0.855297	test's auc: 0.787022
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[349]	train's auc: 0.861721	test's auc: 0.782969
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[286]	train's auc: 0.850082	test's auc: 0.786325
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[456]	train's auc: 0.876827	test's auc: 0.784068


,Train AUC,Test AUC
0,0.8564,0.7842
1,0.8553,0.7870
2,0.8617,0.7830
3,0.8501,0.7863
4,0.8768,0.7841
Avg,0.8601,0.7849


In [58]:
train.to_csv('../data/train_filtered.csv', chunksize=500)

In [ ]:
# https://stats.stackexchange.com/questions/231623/features-selection-why-does-boruta-confirms-all-my-features-as-important/231630#231630